## Webscraping and Applied ML - Project
_Authors_: Alessia SARRITZU, Alberto MARTINELLI

### Step 1: Get hotel data from Foursquare
* Import the necessary libraries
* Define the necessary functions for the API calls and data retrival
* Retrieve and save the data in .csv format

In [25]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests

In [26]:
def get_api_key():
    # Load environment variables from the .env file
    load_dotenv()

    # Get the API key from the environment variable
    api_key = os.getenv('API_KEY')  # Fetch the API key

    if api_key is None:
        print("API key is missing in the .env file!")
        exit()
    return api_key

def convert_json_to_df(data):
    if data:
        flattened_data = []

        # Check if 'results' key exists (Type 1 JSON structure)
        if isinstance(data, dict) and 'results' in data:
            places = data['results']
        # Otherwise, assume it's a list (Type 2 JSON structure)
        else:
            places = data

        # Iterate over each place in 'places' (works for both Type 1 and Type 2)
        for place in places:
            # Extract the required fields based on Type 1 or Type 2 structure
            if 'fsq_id' in place:  # Type 1 (place has 'fsq_id' and 'categories')
                place_data = {
                    'fsq_id': place['fsq_id'],
                    'name': place['name'],
                    'address': place['location']['address'] if 'location' in place else None,
                    'locality': place['location']['locality'] if 'location' in place else None,
                    'country': place['location']['country'] if 'location' in place else None,
                    'formatted_address': place['location']['formatted_address'] if 'location' in place else None,
                    'latitude': place['geocodes']['main']['latitude'] if 'geocodes' in place else None,
                    'longitude': place['geocodes']['main']['longitude'] if 'geocodes' in place else None,
                    'distance': place['distance'] if 'distance' in place else None,
                    'link': place['link'],
                    'categories': [category['name'] for category in place['categories']]  # Extract category names
                }
            elif 'id' in place:  # Type 2 (place has 'id' and 'text')
                place_data = {
                    'id': place['id'],
                    'created_at': place['created_at'],
                    'text': place['text']
                }
            flattened_data.append(place_data)

        # Convert the list of flattened data into a DataFrame
        df = pd.DataFrame(flattened_data)
        return df
    else:
        print("No data available to save.")

def convert_df_to_csv(df, filename):
    # Save the DataFrame to CSV
    df.to_csv(filename, index=False)
    # print("CSV file has been saved as 'output.csv'")

def get_json_data_from_api(url, api_calls):
    api_key = get_api_key()

    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }

    # Make the GET request to the API
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        # print(data)
    else:
        print(f"Error: {response.status_code}")

    api_calls[0] += 1
    return data

In [ ]:
api_calls = [0]

# Get the HOTELS in newyork near 124 Hudson Street in a radiues of 150 meters
url = "https://api.foursquare.com/v3/places/search?query=hotel&ll=40.720276093678535%2C-74.00855578601094&limit=50"
data = get_json_data_from_api(url, api_calls)

df = convert_json_to_df(data)

#for each fsq_id value in the fsq_id column, make an API call to get the tips for that place
# and then add the tips to the dataframe
for fsq_id in df['fsq_id']:
    url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"
    tips = get_json_data_from_api(url, api_calls)

    tips_text = [tip['text'] for tip in tips]  # Get the 'text' from each tip

    # Assign the list of tip texts to the dataframe
    df.loc[df['fsq_id'] == fsq_id, 'tips'] = ', '.join(tips_text)


print(df.head())

convert_df_to_csv(df, 'hotel_reviews_API.csv')
print("CSV file has been saved as 'output.csv'")
print("API calls made:", api_calls[0])

### Step 2: Get hotel data from KAYAK - Web Scraping
* Import the necessary libraries
* Define the necessary function for the Playwright and data retrival
* Retrieve and save the data in .csv format

In [ ]:
import nest_asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def scrape_reviews_with_playwright_async(hotel_url):
    reviews = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Use headless=True for performance
        page = await browser.new_page()
        await page.goto(hotel_url)

        try:
            if await page.locator("button:has-text('Accept')").is_visible():
                await page.locator("button:has-text('Accept')").click()

            hotel_name_element = await page.query_selector("h1.c3xth-hotel-name")
            hotel_name = (await hotel_name_element.inner_text()).strip() if hotel_name_element else "Unknown Hotel"

            await page.wait_for_selector(".acD_-reviews-row-header")

            review_elements = await page.query_selector_all(".acD_")
            for review in review_elements:
                try:
                    rating_element = await review.query_selector(".wdjx-positive")
                    rating = (await rating_element.inner_text()).strip() if rating_element else None

                    score_description_element = await review.query_selector(".acD_-score-description")
                    score_description = (
                        await score_description_element.inner_text()).strip() if score_description_element else None

                    user_name_date_element = await review.query_selector(".acD_-userName")
                    user_name_date = (
                        await user_name_date_element.inner_text()).strip() if user_name_date_element else None

                    pros_element = await review.query_selector(".acD_-pros")
                    pros = (await pros_element.inner_text()).strip() if pros_element else None

                    full_review_element = await review.query_selector("span[id^='showMoreText']")
                    full_review = (await full_review_element.inner_text()).strip() if full_review_element else None

                    reviews.append({
                        "Hotel Name": hotel_name,
                        "Rating": rating,
                        "Score Description": score_description,
                        "User and Date": user_name_date,
                        "Pros": pros,
                        "Full Review": full_review,
                    })
                except Exception as e:
                    print(f"Error processing review: {e}")

        except Exception as e:
            print(f"Error scraping {hotel_url}: {e}")
        finally:
            await browser.close()

    return reviews

async def scrape():
    links_file = "kayak_hotel_links.txt"  # Ensure this file exists
    all_reviews = []

    with open(links_file, 'r') as file:
        hotel_urls = file.read().splitlines()

        for hotel_url in hotel_urls:
            reviews_data = await scrape_reviews_with_playwright_async(hotel_url)

            if reviews_data:
                all_reviews.extend(reviews_data)

    if all_reviews:
        reviews_df = pd.DataFrame(all_reviews)
        reviews_df.to_csv("hotel_reviews_playwright_l.csv", index=False)

# Run the scraper in Jupyter Notebook
await scrape()

### Step 3: Visualize a data preview

In [ ]:
from tabulate import tabulate
import pandas as pd
def display_data_preview(filename):
    pd.set_option('display.max_columns', 200)
    pd.set_option('display.max_rows', 200)
    pd.set_option('display.max_colwidth', 200)

    df = pd.read_csv(filename)
    print("Dataframe preview:")
    print(tabulate(df, headers='keys', tablefmt='psql'))

display_data_preview('hotel_reviews_API.csv')
display_data_preview('hotel_reviews_playwright.csv')

### Step 4: Aggregate API and Scraping reviews for the same hotels in a csv file

In [ ]:
playwright_file = "hotel_reviews_playwright.csv"
playwright_df = pd.read_csv(playwright_file)

# Group and aggregate the 'Full Review' column by 'Hotel Name'
aggregated_reviews = (
    playwright_df.groupby('Hotel Name', sort=False)['Full Review']
        .apply(lambda x: ' | '.join(x))
        .reset_index()
)

aggregated_reviews.rename(columns={'Full Review': 'Aggregated Reviews'}, inplace=True)
aggregated_reviews.to_csv("aggregated_playwright_reviews.csv", index=False)


In [13]:
playwright_file = "aggregated_playwright_reviews.csv"
api_file = "hotel_reviews_API.csv"

playwright_df = pd.read_csv(playwright_file)
api_df = pd.read_csv(api_file)

# Remove specified rows from the API dataset (The data corresponding to the following was not found on Kayak)
excluded_hotels = [
    "Lobby Bar at Ace Hotel New York",
    "The Bar Room at Temple Court",
    "Mr. Purple",
    "The Lobby",
    "Bemelmans Bar"
]
api_df = api_df[~api_df['name'].isin(excluded_hotels)].reset_index(drop=True)

api_df['kayak'] = playwright_df['Aggregated Reviews']
api_df.to_csv("api_playwright_aggregated_reviews.csv", index=False)